In [1]:
from __future__ import annotations

import os
import ssl
import time

import urllib3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait

search_string = '"mine field" -urban -city'
folder = "test"
iter = 0

In [2]:
class Driver:
    def __init__(self: Driver) -> None:
        options = webdriver.ChromeOptions()
        options.add_argument("--remote-debugging-port=8000")
        self.driver = webdriver.Chrome()  # options.add_argument("--headless")
        self.wait = WebDriverWait(self.driver, 10)
        # self.driver.maximize_window()
        self.driver.set_window_size(1920, 1080)
        self.driver.get("https://www.google.com/")

    def setup(self: Driver) -> None:
        self.cookie_validator()

    def cookie_validator(self: Driver) -> None:
        self.wait.until(
            ec.presence_of_element_located((By.CSS_SELECTOR, "#L2AGLb > div")),
        )
        Driver.driver.find_element(By.CSS_SELECTOR, "#L2AGLb > div").click()

    def search(self: Driver, search_string: str) -> None:
        self.driver.get(
            f"https://www.google.com/search?q={search_string}&tbm=isch",
        )

    def get_images(self: Driver) -> list:
        return self.driver.find_elements(
            By.XPATH,
            '//*[@id="islrg"]/div[1]/div/a[1]/div/img',
        )

In [3]:
class Item:
    def __init__(self: Item, url: str, driver: Driver) -> None:
        self.url = url
        self.Driver = driver

    def get_children(self: Item) -> list:
        return Driver.driver.find_elements(
            By.XPATH,
            '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/c-wiz/div/div/div/div[1]/div[3]/div/div/a[1]/div[1]/img',
        )

    def download(self, image_element: list) -> None:
        image_element[0].click()

        global iter
        try:
            print(f"{os.getcwd()}")
            if not os.path.exists(f"./download/{folder}"):
                os.mkdir(f"./download/{folder}")

            http = urllib3.PoolManager(cert_reqs="CERT_NONE")
            response = http.request(
                "GET",
                image_element[0].get_attribute("src"),
                preload_content=False,
            )
            with open(f"./download/{folder}/{iter}-fille.jpg", "wb") as out_file:
                out_file.write(response.data)
            response.release_conn()
            iter += 1
        except Exception as e:
            print(e)
            pass

        self.driver.execute_script("window.history.go(-1)")

In [4]:
Driver = Driver()
Driver.setup()
Driver.search(search_string)
images = Driver.get_images()

In [5]:
def download_image(image_element) -> None:
    global iter
    try:
        print(f"{os.getcwd()}")
        if not os.path.exists(f"./download/{folder}"):
            os.mkdir(f"./download/{folder}")

        http = urllib3.PoolManager(cert_reqs="CERT_NONE")
        response = http.request(
            "GET", image_element[0].get_attribute("src"), preload_content=False
        )
        with open(f"./download/{folder}/{iter}.jpg", "wb") as out_file:
            out_file.write(response.data)
        response.release_conn()
        iter += 1
    except Exception as e:
        print(e)
        pass

In [17]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


def find_elements(xpath):
    """Function to find elements with explicit wait and return them."""
    return WebDriverWait(Driver.driver, 10).until(
        ec.presence_of_all_elements_located((By.XPATH, xpath))
    )


for index, image in enumerate(images):
    try:
        image.click()

        main_image_xpath = '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]'
        main_image_element = find_elements(main_image_xpath)
        # Download main image
        print(main_image_element[0].get_attribute("src"))
        download_image(main_image_element)
        # Process child images
        children_xpath = '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/c-wiz/div/div/div/div[1]/div[3]/div/div/a[1]/div[1]/img'
        children = find_elements(children_xpath)
        print(f"Found {len(children)} child images")
        time.sleep(3)

        for child_id, child in enumerate(children):
            try:
                children = find_elements(children_xpath)
                child = children[child_id]
                child.click()
                print(f"Found {len(children)} child images")
                time.sleep(3)
                # Here you can add code to download child image
                child_image_xpath = '//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]'
                child_image_element = find_elements(main_image_xpath)
                print(child_image_element[0].get_attribute("src"))
                download_image(child_image_element)
                Driver.driver.execute_script("window.history.go(-1)")
            except Exception as e:
                print(f"Error processing child image {child_id}: {e}")
    except Exception as e:
        print(f"Error processing main image {index}: {e}")

https://www.csiro.au/-/media/WordPress/CSIROBlog/AdobeStock_1119463-1024x683.jpeg?mw=800&hash=52E7503FA239B2FD7BB967B1B69A657C
/Users/guillaumedorschner/Library/Mobile Documents/com~apple~CloudDocs/Ecole/4 - ESILV/A4/S7/PI 2/mine-clearance/AI/BotScraper


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.csiro.au'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Found 12 child images
Found 12 child images
https://t3.ftcdn.net/jpg/00/01/11/94/360_F_1119461_vPl91XYaxph4JKqEFldV1ld0OGHIJK.jpg
/Users/guillaumedorschner/Library/Mobile Documents/com~apple~CloudDocs/Ecole/4 - ESILV/A4/S7/PI 2/mine-clearance/AI/BotScraper


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 't3.ftcdn.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Found 12 child images
https://qph.cf2.quoracdn.net/main-qimg-c234203458bc94fb5de7cb3d852a5070
/Users/guillaumedorschner/Library/Mobile Documents/com~apple~CloudDocs/Ecole/4 - ESILV/A4/S7/PI 2/mine-clearance/AI/BotScraper


...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'previews.123rf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Found 12 child images
https://hips.hearstapps.com/hmg-prod/images/5390259-1563221599.jpg?crop=1.00xw:0.756xh;0,0.244xh&resize=1200:*
/Users/guillaumedorschner/Library/Mobile Documents/com~apple~CloudDocs/Ecole/4 - ESILV/A4/S7/PI 2/mine-clearance/AI/BotScraper


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hips.hearstapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Found 12 child images


KeyboardInterrupt: 